In [4]:
import pandas as pd
import numpy as np
df_m = pd.read_csv('Data/MarvelComicCaracters.csv', sep=';')

# Описание данных

- name — оригинальное имя персонажа вселенной Marvel
- align — принадлежность персонажа к доброй/злой/нейтральной стороне
- eye — описание глаз персонажа
- hair — описание волос персонажа
- sex — пол персонажа
- appearances — количество появлений персонажа
- first_appearance — год первого появления персонажа

In [5]:
df_m.head(n=3)

,name,align,eye,hair,sex,appearances,first_appearance
0,Spider-Man (Peter Parker),Добрый персонаж,Карие глаза,Каштановые волосы,Мужской персонаж,4043.0,1962.0
1,Captain America (Steven Rogers),Добрый персонаж,Голубые глаза,Белые волосы,Мужской персонаж,3360.0,1941.0
2,"Wolverine (James \""Logan\"" Howlett)",Нейтральный персонаж,Голубые глаза,Черные волосы,Мужской персонаж,3061.0,1974.0


# Задания

### Сколько пропущенных значений в переменной `appearances`? Выберите один верный ответ.

In [6]:
df_m['appearances'].isna().sum()

1096

### Укажите, верно ли следующее утверждение: наибольший межквартильный размах по переменной `appearances` будет для злых персонажей. Вам понадобятся данные по типу персонажа (переменная `align`), удалите пропущенные значения по столбцу `appearances` и посчитайте требуемую меру разброса.

In [7]:
def get_iqr(a):
    q1 = np.percentile(a, 25)
    q3 = np.percentile(a, 75) 
    return q3 - q1


def get_iqr_outlier_thresholds(df, col, n_iqr):
    q1 = df[col].quantile(0.25)
    q3 = df[col].quantile(0.75)
    iqr = q3 - q1
    
    iqr_outlier_threshold_bottom = q1 - n_iqr * iqr
    iqr_outlier_threshold_up = q3 + n_iqr * iqr
    return iqr_outlier_threshold_bottom, iqr_outlier_threshold_up

In [8]:
df_m \
    .loc[df_m['appearances'].notna(), :] \
    .groupby('align')['appearances'].agg(get_iqr) \
    .sort_values(ascending=False)

align
Добрый персонаж         13.0
Нейтральный персонаж     8.0
Злой персонаж            5.0
Name: appearances, dtype: float64

### Сколько выбросов встречается по переменной `appearances`? Работайте с исходными данными, предварительно удалите пропущенные значения по столбцу `appearances`, используйте для нахождения выбросов межквартильный размах. Выберите один верный ответ.

In [11]:
d_notna = df_m.loc[df_m['appearances'].notna(), :]

d_notna.loc[d_notna['appearances'] > get_iqr_outlier_thresholds(d_notna, 'appearances', 1.5)[1]].shape[0]

1938

### На основе данных, полученных в третьем номере, укажите, верно ли следующее утверждение: наибольшее количество выбросов по переменной `appearances` наблюдается у добрых персонажей.

In [14]:
d_notna \
    .loc[d_notna['appearances'] > get_iqr_outlier_thresholds(d_notna, 'appearances', 1.5)[1]] \
    .pivot_table(values='name', index='align', aggfunc='count', dropna=False) \
    .sort_values('name', ascending=False) \
    .index[0]

'Добрый персонаж'

### Создайте новую переменную, которая будет представлять собой прологарифмированную переменную `appearances`. Сколько выбросов получится по новой переменной? Работайте с исходными данными, предварительно удалите пропущенные значения по новому столбцу, используйте для нахождения выбросов три среднеквадратичных отклонения от среднего. Введите ответ в виде целого числа.

In [27]:
def get_std_outlier_thresholds(df, col, n_std):
    mean, std = df[col].mean(), df[col].std()
    std_outlier_threshold_bottom = mean - n_std * std
    std_outlier_threshold_up = mean + n_std * std
    return std_outlier_threshold_bottom, std_outlier_threshold_up

In [34]:
d_log = df_m.copy()
d_log['appearances_log'] = np.log(d_log['appearances'])
d_log = d_log.loc[d_log['appearances_log'].notna()]

low, high = get_std_outlier_thresholds(d_log, 'appearances_log', 3)
d_log.loc[(d_log['appearances_log'] > high) |
          (d_log['appearances_log'] < low)
          ,:].shape[0]

195

### Сколько выбросов получится по переменной `appearancs`? Работайте с исходными данными, предварительно удалите пропущенные значения по столбцу `appearances`, используйте для нахождения выбросов три среднеквадратичных отклонения от среднего. Введите ответ в виде целого числа.

In [48]:
d_notna = df_m.loc[df_m['appearances'].notna(), :]

low, high = get_std_outlier_thresholds(d_notna, 'appearances', 3)
d_notna.loc[(d_notna['appearances'] < low) | (d_notna['appearances'] > high)].shape[0]

128

### Сколько выбросов получится по переменной `appearancs`, если выбраны только женские персонажи? Работайте с исходными данными, выберите женских персонажей, удалите пропущенные значения по столбцу `appearancs`, используйте для нахождения выбросов z-оценку и три среднеквадратичных отклонения от среднего. Введите ответ в виде целого числа.

In [49]:
d_fem_notna = df_m[(df_m['appearances'].notna()) &
                   (df_m['sex'] == 'Женский персонаж')
                  ]
low, high = get_std_outlier_thresholds(d_fem_notna, 'appearances', 3)
d_fem_notna.loc[(d_fem_notna['appearances'] < low) | (d_fem_notna['appearances'] > high)].shape[0]

48

### Сравните среднее значение по переменной `appearancs` в двух датафреймах — без выбросов, определенных по межквартильному размаху, и без выбросов, определенных по трем среднеквадратичным отклонениям от среднего. Будем считать, что границы датафреймов без выбросов содержат значения верхних и нижних границ, определенных по межквартильному размаху или среднеквадратичному отклонению от среднего. В ответ запишите число (наибольшее среднее из двух), округлите до целого.

In [63]:
d_notna = df_m.loc[df_m['appearances'].notna(), :]
d_notna

std_low, std_high = get_std_outlier_thresholds(d_notna, 'appearances', 3.0)
iqr_low, iqr_high = get_iqr_outlier_thresholds(d_notna, 'appearances', 1.5)

outliers = {
    'std': d_fem_notna.loc[(d_fem_notna['appearances'] <= std_low) | (d_fem_notna['appearances'] >= std_high), 'appearances'].mean(),
    'iqr': d_fem_notna.loc[(d_fem_notna['appearances'] <= iqr_low) | (d_fem_notna['appearances'] >= iqr_high), 'appearances'].mean()
}

round(
    max(
        outliers.values()
    )
)

617

### Укажите, верно ли следующее утверждение: в датафрейме для добрых женских персонажей мода по переменной `hair` не изменится, если удалить выбросы по переменной `appearances`. Работайте с исходными данными, предварительно удалите пропущенные значения по столбцу `appearances`, используйте для нахождения выбросов межквартильный размах.

In [76]:
d_notna = df_m.loc[df_m['appearances'].notna(), :]

iqr_low, iqr_high = get_iqr_outlier_thresholds(d_notna, 'appearances', 1.5)


d_fem_notna.loc[(d_fem_notna['appearances'] >= iqr_low) & (d_fem_notna['appearances'] <= iqr_high), 'hair'].mode().values == df_m['hair'].mode().values

array([ True])

### Выберите только злых персонажей. Сравните медиану по переменной appearances в двух датафреймах — с выбросами и без выбросов, определенных по межквартильному размаху. В ответ запишите число (наибольшее среднее из двух), округлите до целого.

In [83]:
d_u = df_m[df_m['align'] == 'Злой персонаж']
d_u_notna = d_u.loc[d_u['appearances'].notna(), :]

iqr_low, iqr_high = get_iqr_outlier_thresholds(d_u_notna, 'appearances', 1.5)
median = {'d_u': d_u.loc[:, 'appearances'].median(),
          'd_u_notna': d_u_notna.loc[(d_u_notna['appearances'] < iqr_low) | (d_u_notna['appearances'] > iqr_high), 'appearances'].median()
         }
max(median.values())

27.0